In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plot
from langdetect import detect, DetectorFactory
import re   
from google.cloud import translate
import os
import time
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [7]:

stemmer = PorterStemmer()
stop_words=set(stopwords.words('english'))
print(len(stop_words))
print(stop_words)


sample_text="I don't think the service is good, I have a very bad feeling about it"
sample_text = stemmer.stem(sample_text)
tokenized_word=word_tokenize(sample_text)
print(tokenized_word)



179
{'ain', 'hadn', 'themselves', "you'll", 'its', 'm', 'above', 'under', "wouldn't", 'as', "shan't", "mustn't", 'after', 'haven', 'weren', 'few', 'so', 'our', 'no', 'll', 'some', 'up', 'any', 'did', 'about', 'all', 'me', "wasn't", 'be', 'these', 'this', 'should', 'had', 're', 'been', 'there', 'my', 'being', 'the', 'until', 'own', 'y', 'mightn', 'again', 'couldn', 'hers', 'do', 'we', 't', "she's", "won't", 'having', "you'd", 'that', "haven't", 'don', 'not', 'will', 'a', 'at', 'same', 'myself', 'too', 'now', 'has', 'down', 'when', "should've", 'their', 'hasn', 'both', 'shan', 'it', "mightn't", "don't", "needn't", "aren't", 'by', 'why', 'nor', 'was', 'with', 'from', 'further', 'didn', 'ourselves', 'who', 'doing', 'than', "hasn't", "shouldn't", 'which', 'most', 'yourself', 'or', 'itself', 'off', 'theirs', 'such', 've', 'aren', 'ma', 'they', 'you', 'below', 'd', 'once', "you've", 'other', 'what', 'those', 'into', 'each', 'mustn', 'herself', "hadn't", 'but', "that'll", 'because', "weren't",

In [8]:
df = pd.read_excel("data.xlsx")
df.fillna('-')
list_of_comments = df['Verbatim']


In [9]:
# The following lines of code cleans the Touchpoint column of the dataset

df['Touchpoint'] = df['Touchpoint'].str.replace('&lt;item&gt;', '')
df['Touchpoint'] = df['Touchpoint'].str.replace('&lt;/item&gt;', ', ')


In [125]:
# Initialize Translation client
def detect_lang_text(text="saya suka mandi", project_id="data-338918"):
    """Translating Text."""
    start = time.time()

    client = translate.TranslationServiceClient()

    location = "global"

    parent = f"projects/{project_id}/locations/{location}"

    # Translate text from English to French
    # Detail on supported types can be found here:
    # https://cloud.google.com/translate/docs/supported-formats

    if (text):
        lang_detect = client.detect_language(
            content=text,
            parent=parent,
            mime_type="text/plain",  # mime types: text/plain, text/html
        )
        lang_detected = lang_detect.languages[0].language_code
    else:
        lang_detected = "en"
    # Display list of detected languages sorted by detection confidence.
    # The most probable language is first.
    # print(type(text))
    if(not lang_detected):
        lang_detected = "en"
    return lang_detected


In [ ]:
clean_verbatim = []
lang_detected=[]
const=len(list_of_comments)
# for i in list_of_comments:
for i,n in enumerate(list_of_comments):
    cleaned_output = re.sub(r'[^a-zA-z\s]','',str(n))
#     cleaned_output = re.sub(r'[^\w\s]','',str(i))
#     cleaned_output = re.sub(r'ï½','', cleaned_output)
    cleaned_output = cleaned_output.lower()
    print(i)
    detected_lang=detect_lang_text(cleaned_output)
    lang_detected += [detected_lang]
    clean_verbatim +=[cleaned_output]
# df.insert(4, "Verbatim Translated", clean_verbatim, True)
df.insert(4, "Lang Detected", lang_detected, True)
df["Verbatim"]=clean_verbatim
#TODO: Adding a cache or create a new dataset containing the translated values so we dont run the analysis everytime we run the code

In [148]:
df.head(200)
df.to_csv("language_tagged.csv")

,Created,Date,Touchpoint,Rating,Lang Detected,Verbatim,Preferred Bank,Product,ID
0,8/27/2020 4:42:25 PM,8/27/2020 16:42:24,"Branch,",7,ms,talian careline susah dijawab harus diperbaiki,RHB,CASA,bedba74e-1c23-4772-9ca9-6bf81bd1b27f
1,8/27/2020 5:22:49 PM,8/27/2020 17:22:33,"ATM, Cash Deposit Machines,",6,ms,tiada atm atau rhb di jasin melaka,Maybank,CASA,e170ee8d-1287-4f22-b597-c1f2d2b3e2cc
2,8/27/2020 5:50:18 PM,8/27/2020 17:50:17,"RHB Mobile Banking Application, ATM, Cash Depo...",6,en,up grade your apps,Affin Bank CIMB,CASA,feb9fc8d-15c3-407b-961a-e1f09677f3d0
3,8/27/2020 8:12:22 PM,8/27/2020 20:12:19,"Branch, ATM, Cash Deposit Machines,",10,ms,mengekalkan perkhidmatan seperti sekarang dan ...,Rhb bank di petaling jaya ss2.,CASA,22990519-8eb0-4a16-9e07-c9050c50e520
4,8/27/2020 8:14:38 PM,8/27/2020 20:14:36,"RHB Internet Banking,",5,en,hope can improve mobile app factions or link b...,ocbc pub,CASA,d2122a89-2eef-43e0-b8f8-44d399c04498
...,...,...,...,...,...,...,...,...,...
195,8/28/2020 7:58:43 AM,8/28/2020 7:58:45,"Branch, Call centre,",8,ms,perbank internet menyediakan pelbagai perkhidm...,MAYBANK,PF,9250b073-17c1-4a43-98d2-5f7e11874336
196,8/28/2020 8:01:49 AM,8/28/2020 8:1:46,"ATM, Cash Deposit Machines,",10,bs,servis,RHB Bank,PF,79ab2936-8164-401c-861d-ad0eac6407af
197,8/28/2020 8:02:25 AM,8/28/2020 8:2:23,"RHB Internet Banking,",10,en,no comment,No comment,PF,0455052f-e04e-476d-b9a4-dade90d10eb4
198,8/28/2020 9:09:35 AM,8/28/2020 9:9:31,"Branch, Social media, ATM, Cash Deposit Machin...",10,ms,kekalkan khidmat mesra pelanggan,Public Bank,PF,22e8c459-eabd-4128-98d1-3eae51b3a413


In [ ]:
# Initialize Translation client
def translate_text(text="saya suka mandi",lang="en", project_id="data-338918"):
    """Translating Text."""
    start = time.time()

    client = translate.TranslationServiceClient()

    location = "global"

    parent = f"projects/{project_id}/locations/{location}"

    # Translate text from English to French
    # Detail on supported types can be found here:
    # https://cloud.google.com/translate/docs/supported-formats

    # Display list of detected languages sorted by detection confidence.
    # The most probable language is first.
    # print(type(text))
    if lang != "en" and lang !="und" and text :
        response = client.translate_text(
            request={
                "parent": parent,
                "contents": [text],
                "mime_type": "text/plain",  # mime types: text/plain, text/html
                "source_language_code": lang,
                "target_language_code": "en",
            }
        )
            # Display the translation for each input text provided
        translated_text = response.translations[0].translated_text
        end = time.time()
        print(f"{translated_text} ({end-start}s)")
        return translated_text


    return text

In [13]:
# df_lang_detected = pd.read_csv("language_tagged.csv")
# lang_list = df_lang_detected['Lang Detected']
# clean_verbatim = df_lang_detected["Verbatim"]
# translated_verbatim_list = []

# for (i,n) in enumerate(clean_verbatim):
#     print(i)
#     translated_verbatim=translate_text(text=str(n),lang=lang_list[i])
#     translated_verbatim_list+=[translated_verbatim]
# df_lang_detected.insert(4, "Verbatim Translated", translated_verbatim_list, True)

In [12]:
# Initialize Translation client
df_lang_detected.head(200)
df_lang_detected.to_csv("data_translated.csv")

Start of something new

In [160]:
df_lang_detected = pd.read_csv("data_translated.csv")

In [161]:
# CASA is Current Account Savings Account

print("These are unique products that the bank offers to customers:", pd.unique(df['Product']).tolist())

These are unique products that the bank offers to customers: ['CASA', 'Credit Card', 'FD', 'Mortgage', 'HP', 'PF']


"Language Used" column indicates "ms" for Malay language verbatim and "en" for English language verbatim. Undefined language implies verbatim was not particularly useful and can be discarded.

In [163]:
# list of banks in Msia based on https://en.wikipedia.org/wiki/List_of_banks_in_Malaysia
# hong represents hong leong bank, standard represents standard chartered, mbb is abbreviation for maybank since it is common for people to refer to maybank as mbb

list_of_banks = ["maybank", "mbb", "cimb", "public", "rhb", "hong", "ambank", "uob", "rakyat", "ocbc", "hsbc", "islam", "affin", "alliance", "standard", "mbsb", "citibank", "simpanan", "bsn","mualamat", "agro", "agrobank", "al-rajhi", "al rajhi", "pertama"]

In [164]:
df_lang_detected['Preferred Bank'] = df_lang_detected['Preferred Bank'].str.lower()

user_input = df_lang_detected['Preferred Bank']
cleaned_preferred_banks = []
sentence = ""
for i in user_input:
    splitset_i = set(str(i).split())
    intersection_with_banks = list(splitset_i.intersection(list_of_banks))
    if len(intersection_with_banks) > 0:
        for j in intersection_with_banks:
            sentence += j + ", "
    else:
        sentence = "na"

    cleaned_preferred_banks += [sentence]
    sentence = ""


df_lang_detected['Preferred Bank'] = cleaned_preferred_banks


Here, we clean the preferred banks column. Entries from customers that did not clearly state their preferred bank were deemed to be "not applicable", and it is represent in the table with the entry "na". 

In [166]:
df_lang_detected.head()

,Unnamed: 0,Unnamed: 0.1,Created,Date,Touchpoint,Verbatim Translated,Rating,Lang Detected,Verbatim,Preferred Bank,Product,ID
0,0,0,8/27/2020 4:42:25 PM,8/27/2020 16:42:24,"Branch,",the hard -to -answer careline line needs to be...,7,ms,talian careline susah dijawab harus diperbaiki,"rhb,",CASA,bedba74e-1c23-4772-9ca9-6bf81bd1b27f
1,1,1,8/27/2020 5:22:49 PM,8/27/2020 17:22:33,"ATM, Cash Deposit Machines,",there is no atm or rhb in jasin melaka,6,ms,tiada atm atau rhb di jasin melaka,"maybank,",CASA,e170ee8d-1287-4f22-b597-c1f2d2b3e2cc
2,2,2,8/27/2020 5:50:18 PM,8/27/2020 17:50:17,"RHB Mobile Banking Application, ATM, Cash Depo...",up grade your apps,6,en,up grade your apps,"cimb, affin,",CASA,feb9fc8d-15c3-407b-961a-e1f09677f3d0
3,3,3,8/27/2020 8:12:22 PM,8/27/2020 20:12:19,"Branch, ATM, Cash Deposit Machines,",maintain the service as it is now and gift som...,10,ms,mengekalkan perkhidmatan seperti sekarang dan ...,"rhb,",CASA,22990519-8eb0-4a16-9e07-c9050c50e520
4,4,4,8/27/2020 8:14:38 PM,8/27/2020 20:14:36,"RHB Internet Banking,",hope can improve mobile app factions or link b...,5,en,hope can improve mobile app factions or link b...,"ocbc,",CASA,d2122a89-2eef-43e0-b8f8-44d399c04498
